In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.generation import GenerationConfig
import torch
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [2]:
text = """
Dear WidgetCorp,
I am writing to express my deep disappointment with your service regarding my recent order (Order
#98765). I had ordered the "SuperWidget" as a birthday gift for my daughter, Emily, whose birthday
was on March 15th. Despite your website promising delivery within three days, the product arrived
only yesterday, missing her birthday entirely.
I attempted to contact your customer service multiple times for an explanation and to expedite the
delivery, but to no avail. My calls and emails went unanswered, leaving me in a difficult position with
no gift for Emily’s special day.
This experience has left a sour taste, and I am reconsidering future purchases from WidgetCorp. I
expect a prompt explanation and an assurance that such delays will not occur in the future.
Sincerely,
Jane Doe
"""

# 1.8B Q4 Qwen

In [4]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [5]:
!pip install einops transformers_stream_generator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.4-py3-none-any.whl size=12316 sha256=7825866b329df6634f59db1497eb9ad7d97b5bf8f4eb229ce1a931471ae4e746
  Stored in directory: /root/.cache/pip/wheels/47/1d/3c/92d88493ed40c0d9be60a391eb76c9a56e9f9b7542cb789401
Successfully built transformers_stream_generator


In [6]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.9 MB/s eta 0:00:00


In [7]:
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00


In [8]:
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.5/402.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


In [3]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4", #"Qwen/Qwen-7B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()


In [6]:
response, history = model.chat(tokenizer, "Hi", history=None)
print(textwrap.fill(response, width=80))

Hello! How can I help you today? Is there something on your mind that you would
like to discuss or ask about? I'm here to provide information and assistance on
various topics. Just let me know how I can be of help.


In [7]:
prompt = f"""
What was the reason for the customer's disappointment with WidgetCorp?

Separate your answers with line breaks (\n).

Text:
```{text}```
"""

response, history = model.chat(tokenizer, prompt, history=history)
print(textwrap.fill(response, width=80))

The reason for the customer's disappointment with WidgetCorp was their belief
that the SuperWidget was supposed to arrive within three days and that it did
not. Despite contacting customer service multiple times for an explanation and
expediation of the delivery, the customer did not receive a response or
resolution to their concerns. As a result, the customer is considering cutting
their relationship with WidgetCorp and is expressing their disappointment in a
letter to the company.


In [8]:
prompt = f"""
How did WidgetCorp's customer service respond to the customer's inquiries about the late delivery?

Separate your answers with line breaks (\n).

Text:
```{text}```
"""

response, history = model.chat(tokenizer, prompt, history=history)
print(textwrap.fill(response, width=80))

The customer's inquiry regarding the late delivery was not answered by
WidgetCorp's customer service. The customer's attempts to contact the company
were unanswered, leaving them in a difficult situation with no gift for Emily's
special day. As a result, the customer has decided to cut their relationship
with WidgetCorp and is expressing their disappointment in a letter to the
company.


In [9]:
prompt = f"""
What are the entities in the text below:


Text:
```{text}```
"""

response, history = model.chat(tokenizer, prompt, history=history)
print(textwrap.fill(response, width=200))

In the text above, the following entities are mentioned:    * Jane Doe: The customer who wrote the letter   * WidgetCorp: The company that the customer ordered from   * Order #98765: The specific
order number that the customer placed for the "SuperWidget"   * "SuperWidget": A type of widget that the customer ordered as a gift for her daughter Emily   * March 15th: The date Emily's birthday was
* March 20th: The date the widget finally arrived  These are all the entities that appear in the text above.


In [10]:
prompt = f"""
For the text below, perform the following actions:

Summarize the text.


Text:
```{text}```
"""

response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

Jane Doe is disappointed with WidgetCorp's service regarding her recent order
(Order #98765) which did not arrive on time and left her with no gift for
Emily's special day. She has tried contacting customer service but their answers
were unresponsive. Jane is considering not purchasing from WidgetCorp in the
future and is expecting a prompt explanation and an assurance that such delays
will not occur in the future.


In [11]:
prompt = f"""
For the summary you just gave me, translate it to french

"""

response, history = model.chat(tokenizer, prompt, history=history)
print(textwrap.fill(response, width=80))

Madame Jane Doe s'est déçue de l'service de WidgetCorp sur son dernier commande
(Numéro #98765) qui n'a pas été fourni avant la fin de ma journée et qui leaves
Michel sans gift spécial pour sa特定 journée. Elle a effectué plusieurs visites à
la clientèle pour expliquer et améliorer le temps de livraison, mais leur
réponse ne s'est pas abordée. Elle a également fourni des messages téléphoniques
et des e-mails pour être reçus, mais leur réponse ne s'est pas répondue. Madame
Jane Doe est décida de ne pas acheter à partir de WidgetCorp à la suite de cette
expérience mal.contexte et est预期 que ces délais ne se passent plus dans
l'avenir.


In [12]:
prompt = f"""
Translate the French summary of your previous answer back to english

"""

response, history = model.chat(tokenizer, prompt, history=history)
print(textwrap.fill(response, width=100))

Madame Jane Doe was disappointed with WidgetCorp's service regarding her recent order (Order #98765)
which did not arrive before the end of her day and left Michel without a special gift for his
special day. She had contacted the customer service several times for an explanation and improvement
of the delivery, but they answered her questions late. She also sent letters and emails to be
answered but received none. Madame Jane Doe decided not to buy any widgets from WidgetCorp after
this bad experience.
